In [1]:
import numpy as np
import pandas as pd

#Image processing
import skimage.filters
import skimage.io
import skimage.morphology
import skimage.exposure
import skimage.feature

import altair as alt
import bokeh.io

import bootcamp_utils

from behavioral_analysis import segmentation as seg
import tracking

bokeh.io.output_notebook()

Loading BokehJS ...

In [2]:
#Read in images
im_test=skimage.io.imread('toy_data/ant_scepto-04092018133745-6090.tiff')
im_bg=skimage.io.imread('dalotia_data/dalotia_foraging_test-07102018095543-0.tiff')

In [5]:
tracking.process_im(im_test, size_thresh=[200, 1200])

TypeError: 'list' object cannot be interpreted as an integer

In [6]:
#Filter blobs by size. Not too large, not too small.
#Read in images
im_test=skimage.io.imread('toy_data/ant_scepto-04092018133745-6090.tiff')
im_bg=skimage.io.imread('toy_data/col_filed.tiff')

#spec_labels is a list of all specimen [label blob size]
im_test = seg.bg_subtract(im_test, im_bg)
bokeh.io.show(bootcamp_utils.bokeh_imshow(im_test))


a, im_labeled, n_labels=seg.segment(im_test)
spec_labels=[]
im_mask=0*im_labeled

for i in range(0,n_labels):
    blob_size=(im_labeled==i).sum()
    if blob_size>200 and blob_size<1200: #about the size of ants/beetles
        spec_labels.append([i, blob_size])
        im_mask+=(im_labeled==i)
        
im_mask_labeled=skimage.measure.label(im_mask)
properties=skimage.measure.regionprops(im_mask_labeled)
bokeh.io.show(bootcamp_utils.bokeh_imshow(im_labeled))

for x in range(len(properties)):
    print(properties[x].centroid) 
    print(properties[x].area)
    
    
#Match up with euclidean distance but also blob size?

(154.90140845070422, 142.72394366197184)
355
(225.32890365448506, 411.3189368770764)
301
(246.91455160744502, 185.6802030456853)
1182
(268.93708053691273, 407.9001677852349)
1192


In [9]:
import trackBlob
import glob
len(processed_im)

NameError: name 'processed_im' is not defined

In [3]:
all_files=glob.glob("dalotia_data/*")
im_bg=skimage.io.imread('dalotia_data/dalotia_foraging_test-07102018095543-0.tiff')

thresh=[100,1000]

data=pd.DataFrame()
processed_im=[]

for x in all_files:
    im_test=skimage.io.imread(x)
    processed_im.append(trackBlob.processImage(im_test,im_bg,thresh))

data=pd.concat([data, processed_im[0].get('centroid')], axis=1, ignore_index=True)
for j in range(len(processed_im)-1):
    data, processed_im[j+1]=trackBlob.matchBlobs(processed_im[j], processed_im[j+1], data)
    
print(data)

NameError: name 'glob' is not defined

In [12]:
fpaths=['toy_data/ant_scepto-04092018133745-6090.tiff',
       'toy_data/ant_scepto-04092018133745-6094.tiff']
blah=[]

im_bg=skimage.io.imread('toy_data/col_filed.tiff')

thresh=[100, 2500]

data=pd.DataFrame(columns=list('0123'))



AttributeError: module 'numpy' has no attribute 'float128'

In [22]:
oim=blah[0]
nim=blah[1]
db=pd.DataFrame()
size_weight=1
orient_weight=1
xy_weight=1

assert oim.shape==nim.shape
assert type(db)==pd.core.frame.DataFrame

#number of specimen
n_spec=oim.shape[0]

#Matrix of comparison scores 
sizediff=[[np.nan for i in range(n_spec)] for j in range(n_spec)]
orientdiff=[[np.nan for i in range(n_spec)] for j in range(n_spec)]
xydiff=[[np.nan for i in range(n_spec)] for j in range(n_spec)]

#calculate diff in size, orientation, and centroids for every combination of new blob and old blob
for n in range(n_spec):
    for o in range(n_spec):
        sizediff[n][o]=size_weight*(abs(oim.get('size')[o]-nim.get('size')[n]))/oim.get('size')[o]
        orientdiff[n][o]=orient_weight*abs(oim.get('orientation')[o]-nim.get('orientation')[n])
        dist_vec=np.subtract(oim.get('centroid')[o],nim.get('centroid')[n])
        xydiff[n][o]=xy_weight*np.sqrt(np.dot(dist_vec,dist_vec)) #dot product of difference vector

#Add the differences up, want to minimize the sum of differences.
matchProb=np.add(np.add(sizediff,orientdiff),xydiff)

nlabels=np.argsort(np.argmin(matchProb, axis=1))    

ncentroids=[nim.get('centroid')[k] for k in nlabels]

new_tp=pd.Series(data=ncentroids)
return pd.concat([db,new_tp])

0      (150.96604938271605, 143.570987654321)
1     (225.34680134680136, 411.2828282828283)
2    (246.37826086956522, 185.91739130434783)
3     (269.5773195876289, 408.37886597938143)
dtype: object
                                          0
0    (150.96604938271605, 143.570987654321)
1   (225.34680134680136, 411.2828282828283)
2  (246.37826086956522, 185.91739130434783)
3   (269.5773195876289, 408.37886597938143)


In [37]:
db=pd.DataFrame()

db=trackBlob.matchBlobs(blah[0], blah[1], db, size_weight=1, orient_weight=1, xy_weight=1)
db=pd.concat([db,db], axis=1, ignore_index=True)
print(db)
print(db[1][1])

                                          0  \
0    (150.96604938271605, 143.570987654321)   
1   (225.34680134680136, 411.2828282828283)   
2  (246.37826086956522, 185.91739130434783)   
3   (269.5773195876289, 408.37886597938143)   

                                          1  
0    (150.96604938271605, 143.570987654321)  
1   (225.34680134680136, 411.2828282828283)  
2  (246.37826086956522, 185.91739130434783)  
3   (269.5773195876289, 408.37886597938143)  
(225.34680134680136, 411.2828282828283)
